In [1]:
# Run in python console
import nltk; nltk.download('stopwords')

import spacy
import numpy as np
from spacy.lang.fr import French


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
import re
import numpy as np
import pandas as pd
from pprint import pprint

# Gensim
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel

# spacy for lemmatization
import spacy

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim  # don't skip this
import matplotlib.pyplot as plt
%matplotlib inline

     |████████████████████████████████| 1.7MB 4.2MB/s 
  Created wheel for pyLDAvis: filename=pyLDAvis-3.2.2-py2.py3-none-any.whl size=135593 sha256=75e5deb26438d5c8804ca0d78886445698ca966877eb2d7d6aa03bda79015e74
  Stored in directory: /root/.cache/pip/wheels/74/df/b6/97234c8446a43be05c9a8687ee0db1f1b5ade5f27729187eae
Successfully built pyLDAvis


Spacy stopwords

In [3]:
from nltk.corpus import stopwords
stop_words = stopwords.words('french')

In [4]:
# Import Dataset
#df = pd.read_table('https://raw.githubusercontent.com/selva86/datasets/master/newsgroups.json')
#print(df.target_names.unique())
#df.head()

import pandas as pd
import re
df = pd.read_table(
    "/content/drive/MyDrive/Colab Notebooks/S3_NLP/alice.txt", header=None, sep="delimiter")
print(df)

/usr/local/lib/python3.6/dist-packages/pandas/io/parsers.py:767: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  return read_csv(**locals())


                                                      0
0                                     CHAPITRE PREMIER.
1                                   AU FOND DU TERRIER.
2     ALICE, assise auprès de sa sœur sur le gazon, ...
3     de rester là à ne rien faire; une ou deux fois...
4     yeux sur le livre que lisait sa sœur; mais quo...
...                                                 ...
2670  étranges, et peut-être même en leur contant le...
2671  Merveilles du temps jadis: elle la voyait part...
2672  chagrins et trouver plaisir à leurs innocentes...
2673         propre enfance et les heureux jours d'été.
2674                                               FIN.

[2675 rows x 1 columns]


In [5]:
# Convert to list
data = df.values.tolist()

# Remove new line characters
#data = [re.sub('\s+', ' ', sent) for sent in data]

# Remove distracting single quotes
#data = [re.sub("\'", "", sent) for sent in data]

pprint(data[:1])

[['CHAPITRE PREMIER.']]


In [6]:
def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))  # deacc=True removes punctuations

data_words = list(sent_to_words(data))

print(data_words[:100])

[['chapitre', 'premier'], ['au', 'fond', 'du', 'terrier'], ['alice', 'assise', 'aupres', 'de', 'sa', 'sœur', 'sur', 'le', 'gazon', 'commencait', 'ennuyer'], ['de', 'rester', 'la', 'ne', 'rien', 'faire', 'une', 'ou', 'deux', 'fois', 'elle', 'avait', 'jete', 'les'], ['yeux', 'sur', 'le', 'livre', 'que', 'lisait', 'sa', 'sœur', 'mais', 'quoi', 'pas', 'images', 'pas', 'de'], ['dialogues', 'la', 'belle', 'avance', 'pensait', 'alice', 'qu', 'un', 'livre', 'sans', 'images'], ['sans', 'causeries'], ['elle', 'etait', 'mise', 'reflechir', 'tant', 'bien', 'que', 'mal', 'car', 'la', 'chaleur', 'du'], ['jour', 'endormait', 'et', 'la', 'rendait', 'lourde', 'se', 'demandant', 'si', 'le', 'plaisir', 'de'], ['faire', 'une', 'couronne', 'de', 'marguerites', 'valait', 'bien', 'la', 'peine', 'de', 'se', 'lever', 'et'], ['de', 'cueillir', 'les', 'fleurs', 'quand', 'tout', 'coup', 'un', 'lapin', 'blanc', 'aux', 'yeux', 'roses'], ['passa', 'pres', 'elle'], ['il', 'avait', 'rien', 'la', 'de', 'bien', 'etonnan

In [7]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

In [9]:
#data_words_nostops

In [10]:
# Build the bigram and trigram models
bigram = gensim.models.Phrases(data_words, min_count=5, threshold=100) # higher threshold fewer phrases.
trigram = gensim.models.Phrases(bigram[data_words], threshold=100)  

# Faster way to get a sentence clubbed as a trigram/bigram
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

# See trigram example
print(trigram_mod[bigram_mod[data_words[0]]])

/usr/local/lib/python3.6/dist-packages/gensim/models/phrases.py:598: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


['chapitre', 'premier']


In [11]:
# Define functions for stopwords, bigrams, trigrams and lemmatization
def remove_stopwords(texts):
    return [[word for word in simple_preprocess(str(doc)) if word not in stop_words] for doc in texts]

def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent)) 
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [12]:
# Remove Stop Words
data_words_nostops = remove_stopwords(data_words)

# Form Bigrams
data_words_bigrams = make_bigrams(data_words_nostops)

In [13]:
#data_words_bigrams

In [14]:
from spacy.lang.fr import French
spacy.cli.download("fr_core_news_md")

nlp = spacy.load('fr_core_news_md', disable=['parser', 'ner'])

# Do lemmatization keeping only noun, adj, vb, adv
data_lemmatized = lemmatization(data_words_bigrams, allowed_postags=['NOUN', 'ADJ', 'VERB', 'ADV'])

print(data_lemmatized[:100])

✔ Download and installation successful
You can now load the model via spacy.load('fr_core_news_md')
[['chapitre', 'premier'], ['fond', 'terrier'], ['alice', 'assise', 'aupre', 'commencer', 'ennuyer'], ['faire', 'fois', 'jete'], ['oeil', 'livre', 'lire', 'sœur', 'image'], ['dialogue', 'bel', 'avance', 'alice', 'livre', 'image'], ['causerie'], ['eter', 'mise', 'reflechir', 'tant', 'bien', 'mal', 'chaleur'], ['jour', 'endormir', 'lourd', 'demander', 'plaisir'], ['faire', 'couronne', 'marguerite', 'valoir', 'bien', 'peine', 'lever'], ['cueillir', 'fleur', 'quand', 'coup', 'oeil', 'rose'], ['passer', 'pre'], ['bien', 'etonner', 'alice', 'trouver', 'meme'], ['tre', 'extraordinaire', 'entendre', 'parler', 'lapin', 'dire', 'ah'], ['arriverai', 'trop', 'tard', 'apre', 'bien'], ['etonner', 'moment', 'paraître'], ['naturel', 'cependant', 'venir', 'tirer', 'montre'], ['gousset', 'regarder', 'prendre', 'courir', 'plus', 'bel', 'alice'], ['sauter', 'pieds', 'frappee', 'idee', 'jamais', 'voir'], ['la

In [15]:
# Create Dictionary
id2word = corpora.Dictionary(data_lemmatized)

# Create Corpus
texts = data_lemmatized

# Term Document Frequency
corpus = [id2word.doc2bow(text) for text in texts]

# View
print(corpus[:20])

[[(0, 1), (1, 1)], [(2, 1), (3, 1)], [(4, 1), (5, 1), (6, 1), (7, 1), (8, 1)], [(9, 1), (10, 1), (11, 1)], [(12, 1), (13, 1), (14, 1), (15, 1), (16, 1)], [(4, 1), (12, 1), (14, 1), (17, 1), (18, 1), (19, 1)], [(20, 1)], [(21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1)], [(28, 1), (29, 1), (30, 1), (31, 1), (32, 1)], [(9, 1), (21, 1), (33, 1), (34, 1), (35, 1), (36, 1), (37, 1)], [(15, 1), (38, 1), (39, 1), (40, 1), (41, 1), (42, 1)], [(43, 1), (44, 1)], [(4, 1), (21, 1), (45, 1), (46, 1), (47, 1)], [(48, 1), (49, 1), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1)], [(21, 1), (55, 1), (56, 1), (57, 1), (58, 1)], [(45, 1), (59, 1), (60, 1)], [(61, 1), (62, 1), (63, 1), (64, 1), (65, 1)], [(4, 1), (18, 1), (66, 1), (67, 1), (68, 1), (69, 1), (70, 1)], [(71, 1), (72, 1), (73, 1), (74, 1), (75, 1), (76, 1)], [(52, 1), (62, 1), (67, 1), (77, 1)]]


In [22]:
# Build LDA model
%%capture output
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=12, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=300,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)


In [18]:
# Print the Keyword in the 10 topics
#pprint(lda_model.print_topics())
#doc_lda = lda_model[corpus]

In [19]:
# Visualize the topics
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
11     0.163191 -0.074411       1        1  12.743899
4      0.209837 -0.006969       2        1  11.883923
2      0.077842  0.249350       3        1   8.927832
8      0.147817 -0.061237       4        1   8.792329
9     -0.036592 -0.121247       5        1   8.770976
10    -0.007076 -0.180665       6        1   7.753278
1     -0.091594 -0.063012       7        1   7.504571
6     -0.128759 -0.051268       8        1   7.277852
5     -0.121318  0.127942       9        1   7.119096
3      0.092491  0.115031      10        1   6.770091
7     -0.158736  0.015296      11        1   6.713355
0     -0.147102  0.051191      12        1   5.742797, topic_info=            Term        Freq       Total Category  logprob  loglift
9          faire  204.000000  204.000000  Default  30.0000  30.0000
49          dire  398.000000  398.000000  Default  29.0000  29.0000
80          tout  167.000000  167.000000  Default  28.0000  28.0000
4          alice  307.000000  307.000000  Default  27.0000  27.0000
242        autre   78.000000   78.000000  Default  26.0000  26.0000
...          ...         ...         ...      ...      ...      ...
72          idee    7.371984   13.178207  Topic12  -4.4628   2.2763
1941  absolument    4.982197   10.998324  Topic12  -4.8547   2.0654
1741    bourreau    4.314082    7.012072  Topic12  -4.9987   2.3715
93         aller    5.381308   61.067801  Topic12  -4.7776   0.4282
1633      soldat    4.086364   10.494049  Topic12  -5.0529   1.9141

[491 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
115       8  0.975839       abord
1941      9  0.454615  absolument
1941     12  0.454615  absolument
934       5  0.833961    adresser
525      11  0.920378     affaire
...     ...       ...         ...
535       6  0.980989        voix
405       5  0.952139     vouloir
965      12  0.896503        vrai
109       8  0.947196    vraiment
509      11  0.817406         œil

[500 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[12, 5, 3, 9, 10, 11, 2, 7, 6, 4, 8, 1])